In [1]:
from src.data_processing import make_great_dataset,make_future
from src.data_modelisation import model_maker_tester
from src.data_collecting import get_data,csv_maker
import os
from sklearn.ensemble import RandomForestClassifier
from mlflow.client import MlflowClient
import time
import logging 
logging.basicConfig(format='%(asctime)s %(levelname)s:%(name)s:%(message)s', level=logging.INFO)

2024-12-20 19:23:41,015 INFO:root:Mlflow tracking uri : file:mlruns


In [2]:
api_calleur=get_data()
api_calleur()
csv_maker(input_dir = './data/RAW/',data_in_file='data/station_detail_live.csv',output_csv_file='data/station_detail_live.csv',is_in=True,delete=True)    

2024-12-20 19:23:51,038 INFO:root:START GET DATA
2024-12-20 19:23:51,040 INFO:root:CALLING https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/gbfs.json
2024-12-20 19:23:51,049 DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): velib-metropole-opendata.smovengo.cloud:443
2024-12-20 19:23:51,168 DEBUG:urllib3.connectionpool:https://velib-metropole-opendata.smovengo.cloud:443 "GET /opendata/Velib_Metropole/gbfs.json HTTP/11" 200 None
2024-12-20 19:23:51,170 INFO:root:CALLING https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_information.json
2024-12-20 19:23:51,171 DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): velib-metropole-opendata.smovengo.cloud:443
2024-12-20 19:23:51,282 DEBUG:urllib3.connectionpool:https://velib-metropole-opendata.smovengo.cloud:443 "GET /opendata/Velib_Metropole/station_information.json HTTP/11" 200 None
2024-12-20 19:23:51,315 INFO:root:CALLING https://velib-metropole-opendata.smov

In [4]:
####################################  MLFLOW CONF  ####################################

client = MlflowClient(tracking_uri="file:mlruns")
experiment_name = "Velib"
experiment = client.get_experiment_by_name(experiment_name)

########################################################################################

In [5]:
#############################  MODEL MAKER TESTER CONF  #################################

cat_cols=None
to_fit=['last_reported', 'DAY', 'DAY OF WEEK','DAY OF YEAR', 'HOUR', 'MIN', 'numDocks']
to_pred='numBikesAvailable'

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

model_maker_tester_api=model_maker_tester(cat_cols,to_fit,to_pred)

########################################################################################

In [6]:
#################################  MAKE GREAT DATASET CONF  #################################
col_to_drop=['MONTH','YEAR','stationCode','is_installed','station_id',
             'is_returning','is_renting','num_bikes_available','num_docks_available','numDocksAvailable']
key=['last_reported','HOUR','numBikesAvailable']

make_great_dataset_api=make_great_dataset(col_to_drop,key)


In [7]:
#################################  DATA PROCESSING CONF  #####################################

col_sum_1='numBikesAvailable'
col_sum_2='numDocksAvailable'
col_make_delta = 'numBikesAvailable'
col_make_sum='numDocks'
col_make_date='last_reported'
champion_model=RandomForestClassifier()
########################################################################################
flag_drop_file=False


In [8]:
#data loading and processing
df=make_great_dataset_api('__init__/init_data.csv',col_sum_1,col_sum_2,col_make_sum,col_make_date=col_make_date)
df.to_csv(f'data/df_{time.strftime("%y%m%d")}.csv')
last_csv=f'data/df_{time.strftime("%y%m%d")}.csv'
if flag_drop_file : os.remove(last_csv)

2024-12-20 19:23:59,038 DEBUG:root:Data without NA None


In [9]:
#get last model mlflow
#train model
new_champion,score_new_model=model_maker_tester_api(champion_model,'RandmForest',df,f'V0.0.1{time.strftime("%y%m%d")}',param_grid,flg_first=True)
logging.info(f'NEW MODEL SCORE ====== {score_new_model}')
#prediction
prediction=new_champion.predict(make_future(df,'HOUR',to_fit))
logging.info(f'PREDICTION FOR ===== {prediction}')

2024-12-20 19:24:15,644 DEBUG:root:DATA :     numBikesAvailable last_reported  DAY  DAY OF WEEK  DAY OF YEAR  HOUR  MIN  \
0                  12      20241215   15            7          350    20   13   
1                  10      20241215   15            7          350    20    2   
2                  11      20241215   15            7          350    20    2   
3                  11      20241215   15            7          350    21    0   
4                  10      20241215   15            7          350    21    9   
..                ...           ...  ...          ...          ...   ...  ...   
57                  7      20241216   16            1          351    16   47   
58                  8      20241216   16            1          351    16   48   
59                  9      20241216   16            1          351    16   53   
60                 11      20241216   16            1          351    17    0   
61                 10      20241216   16            1          351 